In [1]:
import sys
sys.path.insert(0, '../utils/')

import pandas as pd
import numpy as np

import util_michael as um
from util import *

In [2]:
dtypes = {
    'Area': 'object'
}
Ylabel = 'College_Persistence_Rate'

train = pd.read_csv('./data/cl_train.csv', dtype=dtypes)
test = pd.read_csv('./data/cl_test.csv', dtype=dtypes)

Y = train[Ylabel]
train.drop(columns=[Ylabel], inplace=True)
comb = pd.concat([train, test], ignore_index=True, sort=False)
train.shape, test.shape, comb.shape

((69, 52), (38, 52), (107, 52))

In [3]:
cols = set(comb.columns)

In [4]:
cols = comb.dtypes == 'object'
cat_cols = set(cols.index)

In [5]:
comb[cat_cols].head()

,Area,Five_Year_Graduation_Rate_Percentage_2012,rowid,Overall_Rating,Four_Year_Graduation_Rate_Percentage_2012,Percentage_of_Misconducts_Resulting_in_Suspension_2013,Plan_Growth_Percentile_Grade_10,Collaborative_Teachers,School_Community,PLAN_Attainment_Percentile_Grade_10,...,Five_Year_Graduation_Rate_Percentage_2013,Supportive_Environment,Grade_ACT_Attainment_Percentile_Grade_11,ACT_Spring_2013_Average_Grade_11,Teacher_Attendance_Percentage_2013,Safe,School_Year,One_Year_DropOut_Rate_Percentage_2013,EPAS_Attainment_Percentile,Effective_Leaders
0,21224,1.0,1,ORGANIZED,1.0,1.0,1.0,NEUTRAL,NOT ENOUGH DATA,1.0,...,1.0,NEUTRAL,3.0,13.2,1.0,NEUTRAL,2013 - 2014,3.4,2.0,STRONG
1,21218,2.0,2,PARTIALLY ORGANIZED,2.0,2.0,2.0,WEAK,NOT ENOUGH DATA,2.0,...,2.0,NOT ENOUGH DATA,17.0,16.3,2.0,NOT ENOUGH DATA,2013 - 2014,1.9,9.0,NEUTRAL
2,21228,3.0,3,ORGANIZED,3.0,3.0,3.0,NOT ENOUGH DATA,NOT ENOUGH DATA,3.0,...,3.0,NEUTRAL,77.0,22.1,3.0,STRONG,2013 - 2014,1.9,77.0,NOT ENOUGH DATA
3,21242,4.0,4,NOT YET ORGANIZED,4.0,4.0,4.0,WEAK,NEUTRAL,4.0,...,4.0,NOT ENOUGH DATA,4.0,13.4,4.0,NOT ENOUGH DATA,2013 - 2014,0.8,3.0,WEAK
4,21240,5.0,5,PARTIALLY ORGANIZED,5.0,5.0,5.0,WEAK,STRONG,5.0,...,5.0,NEUTRAL,14.0,15.9,5.0,WEAK,2013 - 2014,8.8,15.0,WEAK


In [6]:
for cc in cat_cols:
    print('--', cc, '--')
    print(comb[cc].unique())

-- Area --
['21224' '21218' '21228' '21242' '21240' '21246' '21304' '21234' '21210'
 '21250' '21238' '21268' '21298' '21236' '21278' '21274' '21264' '21226'
 '21306' '21248' '21244' '21288' '21230' '21286' '21252' '21256' '21232'
 '21282' '21284' '21276' '21654' '21280' '21254' '21216' '21258' '21318'
 '21320' '21272' '21262' '21310' '21214' '21220' '21302' '21294']
-- Five_Year_Graduation_Rate_Percentage_2012 --
[ 1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17. 18.
 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35. 36.
 37. 38. 39. 40. 41. 42. 43. 44. 45. 46. 47. 48. 49. 50. 51. 52. 53. 54.
 55. 56. 57. 58. 59. 60. 61. 62. 63. 64. 65. 66. 67. 68. 69.]
-- rowid --
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48
 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69]
-- Overall_Rating --
['ORGANIZED' 'PARTIALLY ORGANIZED' 'NOT YET ORGANIZED'
 'M

In [7]:
cols_to_drop = ['School_Year']
cols_to_int = [
    'Freshmen_on_Track_Rate_Percentage_2012', 'Freshmen_on_Track_Rate_Percentage_2013',
    'CPS_Performance_Policy_Status', 'Percentage_of_Misconducts_Resulting_in_Suspension_2012',
    'CPS_Performance_Policy_Level', 'PLAN_Attainment_Percentile_Grade_10', 
    'EPAS_Attainment_Percentile', 'Teacher_Attendance_Percentage_2012', 
    'Average_Length_of_Suspensions_2012', 'Probation_Length', 'Five_Year_Graduation_Rate_Percentage_2012',
    'Suspensions_Per_100_2012', 'EXPLORE_Attainment_Percentile_Grade_9', 'PLAN_Fall_2012_Average_Grade_10',
    'Average_Length_of_Suspensions_2013', 'EPAS_Growth_Percentile', 'Teacher_Attendance_Percentage_2013',
    'Suspensions_Per_100_2013', 'Four_Year_Graduation_Rate_Percentage_2013', 'Grade_ACT_Attainment_Percentile_Grade_11',
    'ACT_Growth_Percentile_Grade_11', 'Five_Year_Graduation_Rate_Percentage_2013', 'Percentage_of_Misconducts_Resulting_in_Suspension_2013',
    'EXPLORE_Spring_2013_Average_Grade_10', 'Four_Year_Graduation_Rate_Percentage_2012',
    'EXPLORE_Fall_2011_Average_Grade_9', 'Plan_Growth_Percentile_Grade_10', 'EXPLORE_Growth_Percentile_Grade_9',
    'EXPLORE_Spring_2013_Average_Grade_9'
    
]
cols_to_float = [
    'Teacher_Response_Rate', 'Student_Attendance_Percentage_2013',
    'One_Year_DropOut_Rate_Percentage_2012', 'ACT_Spring_2013_Average_Grade_11',
    'One_Year_DropOut_Rate_Percentage_2013', 'Student_Response_Rate', 'Student_Attendance_Percentage_2012',
    
]
cols_for_labels = [
    'Area', 'Quality_of_Facilities', 'Overall_Rating', 'Collaborative_Teachers', 'Involved_Family',
    'Parent_Teacher_Partnership', 'Ambitious_Instruction', 'Effective_Leaders', 'Safe',
    'Creative_Schools_Certification', 'School_Community', 'Healthy_Schools_Certification', 
    'Supportive_Environment', 
]

In [8]:
comb.drop(columns=cols_to_drop, inplace=True)

In [9]:
for cint in cols_to_int:
    comb[cint].astype(int, copy=False)

In [10]:
for cint in cols_to_float:
    comb[cint].astype(float, copy=False)

In [11]:
comb[cols_for_labels].head()

,Area,Quality_of_Facilities,Overall_Rating,Collaborative_Teachers,Involved_Family,Parent_Teacher_Partnership,Ambitious_Instruction,Effective_Leaders,Safe,Creative_Schools_Certification,School_Community,Healthy_Schools_Certification,Supportive_Environment
0,21224,NOT ENOUGH DATA,ORGANIZED,NEUTRAL,STRONG,NOT ENOUGH DATA,NEUTRAL,STRONG,NEUTRAL,INCOMPLETE DATA,NOT ENOUGH DATA,NOT CERTIFIED,NEUTRAL
1,21218,NOT ENOUGH DATA,PARTIALLY ORGANIZED,WEAK,NEUTRAL,NOT ENOUGH DATA,NOT ENOUGH DATA,NEUTRAL,NOT ENOUGH DATA,INCOMPLETE DATA,NOT ENOUGH DATA,NOT CERTIFIED,NOT ENOUGH DATA
2,21228,NOT ENOUGH DATA,ORGANIZED,NOT ENOUGH DATA,NOT ENOUGH DATA,NOT ENOUGH DATA,STRONG,NOT ENOUGH DATA,STRONG,EXCELLING,NOT ENOUGH DATA,NOT CERTIFIED,NEUTRAL
3,21242,WEAK,NOT YET ORGANIZED,WEAK,WEAK,WEAK,NOT ENOUGH DATA,WEAK,NOT ENOUGH DATA,EMERGING,NEUTRAL,NOT CERTIFIED,NOT ENOUGH DATA
4,21240,NEUTRAL,PARTIALLY ORGANIZED,WEAK,NEUTRAL,STRONG,STRONG,WEAK,WEAK,INCOMPLETE DATA,STRONG,NOT CERTIFIED,NEUTRAL


In [12]:
comb['Area'].unique()

array(['21224', '21218', '21228', '21242', '21240', '21246', '21304',
       '21234', '21210', '21250', '21238', '21268', '21298', '21236',
       '21278', '21274', '21264', '21226', '21306', '21248', '21244',
       '21288', '21230', '21286', '21252', '21256', '21232', '21282',
       '21284', '21276', '21654', '21280', '21254', '21216', '21258',
       '21318', '21320', '21272', '21262', '21310', '21214', '21220',
       '21302', '21294'], dtype=object)

In [13]:
dummy = pd.get_dummies(comb, columns=cols_for_labels)
comb.shape, dummy.shape

((107, 51), (107, 148))

In [14]:
dummy.dtypes.unique()

array([dtype('int64'), dtype('float64'), dtype('uint8')], dtype=object)

In [15]:
dummyc = dummy.copy()
dummyc[Ylabel] = Y

In [16]:
um.print_weak_correlations(dummyc, Ylabel)

rowid :  0.08791347263663826
unique_id :  -0.24852519030369635
CPS_Performance_Policy_Level :  0.08791347263663826
CPS_Performance_Policy_Status :  0.08791347263663826
Probation_Length :  0.08791347263663826
Student_Response_Rate :  -0.10613715967338065
Teacher_Response_Rate :  -0.3554546680026131
EPAS_Growth_Percentile :  0.08791347263663826
EXPLORE_Growth_Percentile_Grade_9 :  0.08791347263663826
Plan_Growth_Percentile_Grade_10 :  0.08791347263663826
ACT_Growth_Percentile_Grade_11 :  0.08791347263663826
EXPLORE_Attainment_Percentile_Grade_9 :  0.08791347263663826
PLAN_Attainment_Percentile_Grade_10 :  0.08791347263663826
EXPLORE_Spring_2013_Average_Grade_9 :  0.08791347263663826
EXPLORE_Spring_2013_Average_Grade_10 :  0.08791347263663826
EXPLORE_Fall_2011_Average_Grade_9 :  0.08791347263663826
PLAN_Fall_2012_Average_Grade_10 :  0.08791347263663826
Freshmen_on_Track_Rate_Percentage_2013 :  0.08791347263663826
Freshmen_on_Track_Rate_Percentage_2012 :  0.08791347263663826
Four_Year_Grad

In [17]:
brk = train.shape[0]

In [18]:
dummytrain = dummy.iloc[0:brk, :].copy()
dummytest = dummy.iloc[brk:, :].copy()
dummytrain.shape, dummytest.shape, Y.shape

((69, 148), (38, 148), (69,))

# Remove weak correlation < 0.1

In [19]:
dummytrain, dummytest = um.remove_weak_correlations(dummytrain, dummytest, Ylabel, Y, skip=['unique_id'])

In [20]:
dummytrain.shape, dummytest.shape

((69, 57), (38, 57))

In [21]:
dummytrain[Ylabel] = Y
output = pd.concat([dummytrain, dummytest], axis=0, ignore_index=True, sort=False)
output.shape

(107, 58)

# Save file

In [22]:
break_and_save_df(output, brk, 'fill', Y, Ylabel)

../utils/util.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  traindata[Ylabel] = Y
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
